# Analyzing the two predictions and how they connect

In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
def read_data(path):

    # Initialize an empty list to store DataFrames
    dfs = []

    # Walk through the directory
    for subdir, _, _ in os.walk(path):
        # Find all .tsv files in the current directory
        for file in glob.glob(os.path.join(subdir, '*.tsv')):
            # Read the file into a DataFrame
            df = pd.read_csv(file, sep='\t')
            print(f'Read {file} succesfully.', flush=True)
            # Append the DataFrame to the list
            dfs.append(df)

    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df.reset_index(drop=True, inplace=True)

    # Now `combined_df` contains data from all .tsv files
    print("All data read.", flush=True)
    return combined_df

#df = pd.read_csv("/scratch/project_2009199/register-vs-genre/results/en_1/020824_large_multiL-large_03_04.tsv", sep='\t')
df = read_data("/scratch/project_2009199/register-vs-genre/results/")

labels_all_hierarchy_with_other = {
    #"MT": ["MT"],
    "LY": ["LY"],
    "SP": ["SP", "it", "os"],
    "ID": ["ID"],
    "NA": ["NA", "ne", "sr", "nb", "on"],
    "HI": ["HI", "re", "oh"],
    "IN": ["IN", "en", "ra", "dtp", "fi", "lt", "oi"],
    "OP": ["OP", "rv", "ob", "rs", "av", "oo"],
    "IP": ["IP", "ds", "ed", "oe"],
}
df["register_prediction"] = df["register_prediction"].apply(lambda x: eval(x))
df["genre_prediction"] = df["genre_prediction"].apply(lambda x: eval(x))

Read /scratch/project_2009199/register-vs-genre/results/en_7/020824_large_multiL-large_03_04.tsv succesfully.
Read /scratch/project_2009199/register-vs-genre/results/en_9/020824_large_multiL-large_03_04.tsv succesfully.
Read /scratch/project_2009199/register-vs-genre/results/en_5/020824_large_multiL-large_03_04.tsv succesfully.
Read /scratch/project_2009199/register-vs-genre/results/en_8/020824_large_multiL-large_03_04.tsv succesfully.
Read /scratch/project_2009199/register-vs-genre/results/en_2/020824_large_multiL-large_03_04.tsv succesfully.
Read /scratch/project_2009199/register-vs-genre/results/en_3/020824_large_multiL-large_03_04.tsv succesfully.
Read /scratch/project_2009199/register-vs-genre/results/en_6/020824_large_multiL-large_03_04.tsv succesfully.
Read /scratch/project_2009199/register-vs-genre/results/en_1/020824_large_multiL-large_03_04.tsv succesfully.
Read /scratch/project_2009199/register-vs-genre/results/en_4/020824_large_multiL-large_03_04.tsv succesfully.
All data r

In [3]:
print(len(df))

1001587


In [4]:
pd.set_option('max_colwidth', 800)

In [5]:
label, counts = np.unique(df.genre_prediction, return_counts=True)
for l,c in zip(label, counts):
    print(f'{l}: {c}')
print("")
label, counts = np.unique(df.register_prediction, return_counts=True)
for l,c in zip(label, counts):
    print(f'{l}: {c}')

['Cookbooks, Food & Wine']: 10165
['Cookbooks, Food & Wine', 'Engineering & Transportation']: 43
['Cookbooks, Food & Wine', 'Engineering & Transportation', 'Literature & Fiction', 'Medicine & Health Sciences', 'Politics & Social Sciences', 'Science & Math']: 2
['Cookbooks, Food & Wine', 'Literature & Fiction']: 1
['Cookbooks, Food & Wine', 'Medicine & Health Sciences']: 1
['Cookbooks, Food & Wine', 'Science & Math']: 8
['Engineering & Transportation']: 123847
['Engineering & Transportation', 'Literature & Fiction']: 51
['Engineering & Transportation', 'Literature & Fiction', 'Medicine & Health Sciences', 'Politics & Social Sciences', 'Science & Math']: 1
['Engineering & Transportation', 'Literature & Fiction', 'Politics & Social Sciences']: 1
['Engineering & Transportation', 'Literature & Fiction', 'Politics & Social Sciences', 'Science & Math']: 1
['Engineering & Transportation', 'Medicine & Health Sciences']: 80
['Engineering & Transportation', 'Politics & Social Sciences']: 3623
['E

In [6]:
# Group by 'col1' and 'col2' and count occurrences
from ast import literal_eval
#df2 = pd.DataFrame()
#df2['genre_prediction'] = df_all['genre_prediction'].apply(literal_eval)
#df2['register_prediction'] = df_all['register_prediction'].apply(literal_eval)


df_combinations = df.explode("register_prediction").explode("genre_prediction")
df = df.explode("genre_prediction")


comb_counts = df_combinations.groupby(['register_prediction', 'genre_prediction']).size().reset_index(name='count')

# Sort the counts in descending order
sorted_comb_counts = comb_counts.sort_values(by='count', ascending=False)

display(sorted_comb_counts.head(10))
display(sorted_comb_counts.tail(10))


,register_prediction,genre_prediction,count
16,IN,Literature & Fiction,190734
18,IN,None,154246
19,IN,Politics & Social Sciences,134903
44,NA,Literature & Fiction,113824
15,IN,Engineering & Transportation,99490
86,dtp,Literature & Fiction,93833
47,NA,Politics & Social Sciences,93575
88,dtp,None,71204
135,ne,Politics & Social Sciences,66902
23,IP,Literature & Fiction,62779


,register_prediction,genre_prediction,count
39,MT,None,2
38,MT,Medicine & Health Sciences,2
28,LY,"Cookbooks, Food & Wine",2
91,ed,Engineering & Transportation,1
93,ed,Medicine & Health Sciences,1
94,ed,Politics & Social Sciences,1
95,ed,Science & Math,1
158,rs,"Cookbooks, Food & Wine",1
157,re,Science & Math,1
35,MT,"Cookbooks, Food & Wine",1


In [8]:
df_weird = df[df.genre_prediction=="Politics & Social Sciences"]

df_weird = df_weird[df_weird['register_prediction'].apply(lambda x: "LY" in x)]  
print(len(df_weird))
display(df_weird["text"].head(n=7))

28


94048     My shame He's taken away My pain is healed in His name I believe, I believe My Redeemer Lives: Verse 2You lift my burden I'll rise with You I'm dancing on this mountaintop to see Your kingdom come My Redeemer Lives: Bridge 1You lift my burden I'll rise with You I'm dancing on this mountaintop To see Your kingdom come My Redeemer Lives: Bridge 1You Lifted Me Out. Whoa, Whoa, Whoa, Whoa… You turned my way, You heard my cry You turned my mourning into shouting Sorrow may last for a night But with.…more than just a music. A people, a scene, a culture, and a circuit through which information and ideas travel… Politics & DIY Anarchy Genres - Movements. - ppt download…more than just a music. A people, a scene, a culture, and a circuit through which information and ideas travel… Politics & DIY...
135120    If the members of the Michigan Conference Executive Committee Could be blindfolded And transported somehow, Unseeing, Across time and space; If they could land, Unwittingly, In the

In [9]:
df_all = df

## Engineering & Transportation

In [10]:
label = 'Engineering & Transportation'
df = df_all[df_all['genre_prediction'].apply(lambda x: label in x)]
df = df.sample(frac = 1)  # shuffle
df.head(30)

,Unnamed: 0,id,labels,text,register_prediction,genre_prediction
525649,74675,9050986,['NA'],"A fireball goes up at the site of an oil train derailment in Casselton, ND, on Dec. 30, 2013. (AP Photo/Bruce Crummy)The oil boom underway in North Dakota has delivered jobs to local economies and helped bring the United States to the brink of being a net energy exporter for the first time in generations.But moving that oil to the few refineries with the capacity to process it is presenting a new danger to towns and cities nationwide — a danger many appear only dimly aware of and are ill equipped to handle.Much of North Dakota’s oil is being transported by rail, rather than through pipelines, which are the safest way to move crude. Tank carloads of crude are up 50 percent this year from last. Using rail networks has saved the oil and gas industry the time and capital it takes to build ...","[NA, ne]",Engineering & Transportation
818030,34159,1724830,['IN'],HO Scale Plastic parts are precision moulded in appropriate colours - checker players and window glazing included\n,"[IN, IP, dtp, ds]",Engineering & Transportation
863250,79379,3991998,['IP'],Whether you need to print fixed barcodes or automate barcode printing by fetching data from a database label flow barcode software has it all / label flow barcode software also includes an integrated database editor and advanced data filtering tools | label flow barcode software easily interfaces with nearly all thermal and barcode label printers so you don't need to worry about device compatibility Label flow barcode ... Lire la suite\n,"[IN, IP, dtp, ds]",Engineering & Transportation
610366,50991,13304419,['IN'],"Horizon 2020, the multi-annual programme for research and innovation of the European Union (EU), is launching two new funding opportunities for thermal storage projects to support the implementation of the MI Innovation Challenge 7: Affordable Heating and Cooling of Buildings.The two new Horizon 2020 Calls for Proposals, promoting collaborative research and innovation activities, specifically reference the involvement of non-EU Mission Innovation members. They address:The development of compact thermal energy storage for electricity load shifting (LC-SC3-ES-6-2019 – Opening date: 05 September 2018 – Deadline: 05 February 2019)The demonstration of high density, integrated thermal storage based on thermochemical materials (LC-EEB-05-2019-20 – Opening date: 16 October 2018 – Deadline: 21 ...","[IN, dtp]",Engineering & Transportation
557504,106530,10647223,['NA'],"ISRO is expected to launch Cartosat-2 satellite with 30 nanosatellites by mid-December - Geospatial WorldISRO is now planning to launch a remote sensing satellite of Cartosat-2 series along with 30 nanosatellites of foreign countries by mid-December.India: Trying to reclaim its name once again post the failure of IRNSS-1H, ISRO is now planning to launch a remote sensing satellite of Cartosat-2 series along with 30 nanosatellites of foreign countries by mid-December.The space agency is getting ready for the Cartosat-2 series satellite mission in mid-December from the Satish Dhawan Space Centre or Sriharikota High Altitude Range (SHAR), an ISRO official told in an interview to news agency PTI.“Isro will be busy in launching a series of satellites from December onwards. We are targeting t...","[NA, ne]",Engineering & Transportation
276477,51073,24496783,['IN'],Fully-coupled 1D model of mobile-bed alluvial hydraulics with a closure drawn from Rossiter modes resonance conceptsCoupled estimation of the energy slope and associated sand-silt transport during high stream power events in alluvial riversWater management in Brussels: knowledge gained from the long time data series of the Flowbru regional telemetry systemA new closure methodology for 1D fully coupled models of mobile-bed alluvial hydraulics: Application to silt transport in the Lower Yellow RiverData-mining analysis of in-sewer infiltration patterns: seasonal characteristic

## Politics & Social Sciences

In [11]:
label = 'Politics & Social Sciences'
df = df_all[df_all['genre_prediction'].apply(lambda x: label in x)]
df = df.sample(frac = 1)  # shuffle
df.head(20)

,Unnamed: 0,id,labels,text,register_prediction,genre_prediction
523858,72884,8963199,['OP'],"Home > Funds > Investment trusts > MoneyWeek investment trust portfolio > MoneyWeek portfolio of investment trusts – October 2013 updateMerryn Somerset Webb reviews our portfolio of investment trusts. What should you buy now, and which laggards should you drop?By now you’ll have figured out that when I said I would review our investment trust portfolio every six months, I meant it not as a firm forecast but more as ‘forward guidance’. Just as Mark Carney might start to have a think about UK interest rates when unemployment falls to 7%, I start to look at our portfolio every six months. Neither of us promise to actually do anything. That’s why it has been nearly nine months since I covered the portfolio.Back then, we made no changes. If you had split your cash equally between the trusts...","[OP, ob]",Politics & Social Sciences
486091,35117,7074296,['IN'],"This list identifies where ImAbhishekPatel stands on popular issues. The choices are limited to the following:If you are logged in, you will also see green or red bullets next to each issue, which indicate whether you agree or disagree with ImAbhishekPatel on that particular issue.You can also click each issue to find other members that agree with ImAbhishekPatel's position on the issue.Until 2014 Jolande Withuis (b. 1949) was a researcher affiliated with the Netherlands Institute for War Documentation (NIOD). She was a columnist for Opzij for many years and currently writes for NRC Handelsblad, Trouw and de Volkskrant. In 2012 Withuis was one of the interviewees in the television programme Zomergasten (Summer Guests). De Bezige Bij has published Erkenning (Recognition, 2002), Na het k...",[IN],Politics & Social Sciences
692952,21662,28627852,['IN'],"VPS could be seen being a hybrid of shared server and committed bodily servers, retaining the easiest of the two. VPS offers all the advantages of a committed server without the large charges that generally arrive having a dedicated server. Put simply, VPS hosting enables you to use focused internet hosting-like services in a minimize cost.Managed VPS hosting providers are alike in quite a lot of ways to hosting that is certainly shared. Shared web hosting offers may provide endless bandwidth likewise but you could potentially secure a hidden charge in the event you pass a particular threshold. All their programs offer you unlimited bandwidth.Hence, VPS is the smartest option. VPS has Each and every on the contains a committed server has and It truly is appreciably considerably less in...",[IN],Politics & Social Sciences
280241,54837,24684458,['NA'],"ALTHOUGH the Hai-O group started as a Chinese company, not many people are aware that about 95% of Hai-O Marketing Sdn Bhd’s distributors from its direct selling division are Malays.“It’s a reflection that we don’t regard ourselves as a Chinese company, but a truly Malaysian company that provides employment to every race,” said Hai-O group chairman Tan Sri Osman S. Cassim.As an extension of its multi-racial philosophy and support for charity, Hai-O Marketing Sdn Bhd will be sponsoring RM15,000 worth of the Mystical Steppes: Along the Silk Road show tickets for several Malay shelter homes in the Klang Valley.Gift of tickets: Che Tom admiring one of the costumes to be worn by the Mystical Steppes dancers. With her are Hai-O group managing director Tan Kai Hee (left), Osman (right) while ...","[NA, ne]",Politics & Social Sciences
16358,16358,33982657,['IN'],"The first week of October is Mental Illness Awareness Week. This Mental Health Ministries Update will highlight resources we hope you will find helpful during this week and in all future efforts to work against the stigma and shame still associated with these no-fault illness of the brain.The National Day of Prayer for Mental Illness Recovery and Understanding is the Tuesday of Mental Illness Awareness Week. This year that date is October 4.

## Literature & Fiction

In [12]:
label = 'Literature & Fiction'
df = df_all[df_all['genre_prediction'].apply(lambda x: label in x)]
df = df.sample(frac = 1)  # shuffle
df.head(30)

,Unnamed: 0,id,labels,text,register_prediction,genre_prediction
742285,70995,31102103,['IN'],"Welcome to the Critique area. It is assumed if you post an image here, you want true honest critiques from other members of this forum. Please post some background/location on the image, and what you were trying to display, or say. Please add your EXIF information so those who offer critique will know your camera settings and maybe not only critique but offer shooting suggestions.Source : A thesis submitted in partial fulfillment of the requirements for the Doctor of Philosophy degree in American Studies in the Graduate College of The University of Iowa, December 2007My dissertation is a foray into blues music’s intricate web of racial taxonomies, an aspect that has been neglected by most existing studies of the genre. In particular, I am interested in significant changes that took pla...",[ID],Literature & Fiction
40066,40066,35159809,[],"The Vintage Polarquilt is a tailored quilted jacket with a fashionable slim fit and a luxurious fleece-fur lining from the women . Buy Now You Will Enjoy A Big Discount. You Do Not Disappointed.Slide Black Taos Sandal Women's Reward So far, the actual charges the special counsel has brought fall into three main buckets:The Trump campaign’s digital operation: Mueller’s team interviewed people involved with Trump’s digital team with an eye toward understanding whether they’d coordinated with Russian online efforts, Yahoo News and ABC News have reportedDots Heel Buckle Women High Ankle White Toe 8 Sandals FSJ Open Black Size Straps Sandals xtAnq6.\n",[None],Literature & Fiction
679926,8636,27975320,['OP'],"Plot Summary: ""Strip Search follows several parallel stories examining personal freedoms vs. national security in the aftermath of 9/11; two main subplots involve an American woman detained in China and an Arab man detained in New York City."" This is a very short but somewhat complex movie that manipulates the viewer so that you can see how two different people are treated in two different countries. We start out feeling sorry for the American woman who is treated poorly in China but not the the Chinese man treated poorly in America. Clearly, in the end we understand that our prejudice clouds our judgment. I know this all sounds kind of deep but the movie is far more enjoyable than my review. Maggie Gyllenhaal plays the American ""damsel in distress"" who is stripped and interrogated and...","[OP, rv]",Literature & Fiction
324808,99404,26915416,['IN'],"Looking for a relationship, but that does not mean that I am not open for dates, coffees, drinks, fun...\n",[ID],Literature & Fiction
560422,1047,10793366,['OP'],"NAILS recorded an album full of madness this year. Sharp riffs, morbid vocals and furious pace. It is made for cleaning of your brain! The music here is like the world after the apocalypse. It has left only a few devoted. The rest died in endless cramps. Brutality, grind passages and absolute chaos. These are NAILS. I would recommend this CD to everyone who wants to be cut up well. The news is like a punch in the face. It will defeat and crush you! Very well!\n","[OP, IP, ds]",Literature & Fiction
109168,109168,38613087,[],"Overstudies off Pieter's road, blamed his ascendant. arrested Jessey threw margins beautifully circumstanced. Comedian Leonhard phenomenally monitors carbon. Vail friends concrete and artificial their apprentices gradate or somnambulate forwhy. He managed to get Antonius to receive, his relays in the afternoon. Surplus and spent Urbain around her nerved or feminized at the baden baden dancing opportune moment. Sulfur clay turkey-trot, she criticizes very mobs. The parotic robot considers it a scroop crime muscularly. Regret sent to the sky that trembles for more time? Nippy and Excatory Skipper elasticates its subminiaturized pulpitum and sinusoidal braids. Ricrable tea that feeds his insecurities declining ontogenetically? the coral and the melodious Hagan leihmutter si

## Cookbooks, Food & Wine

In [13]:
label = 'Cookbooks, Food & Wine'
df = df_all[df_all['genre_prediction'].apply(lambda x: label in x)]
df = df.sample(frac = 1)  # shuffle
df.head(30)

,Unnamed: 0,id,labels,text,register_prediction,genre_prediction
556503,105529,10597232,['HI'],"Preheat rack for outdoor grill or broiler. In large bowl, combine mushrooms and olive oil. Toss to coat. In small bowl, combine sausage, parsley, lemon juice and garlic. Fill mushroom caps with sausage mixture. Place on vegetable grilling rack. Grill until mushrooms are tender and browned, about 15 minutes. This recipe yields 16 appetizers.\n","[HI, re]","Cookbooks, Food & Wine"
107728,107728,38541338,['HI'],"Extremely subtle and very delicate, this special dish, called Buddha's Delight because it's completely vegetarian, is all about texture. The biggest challenge in making it is finding the right ingredients, but the reward is worth the effort. Prepared with fresh vegetables, this recipe is sublime. If you can't find them fresh, don't be tempted to use canned (frozen bamboo shoots and ginkgo nuts are acceptable, however). Traditional Buddha's Delight doesn't call for garlic, but we find it makes all the difference. The recipe also serves 4 as a fabulous vegetarian main course.2 to 3 oz very thin bean thread noodles (2 small skeins; also known as cellophane, glass, or mung bean noodles)Soak mushrooms in 5 cups boiling-hot water in a bowl, keeping them submerged with a small plate and turni...","[HI, re]","Cookbooks, Food & Wine"
150784,36994,46335544,['NA'],YUMMO - 365 Days of Eating and Being Fed : July 25th - BBQd Whole Herb Stuffed Salmon with Basil Butter - Salsa Verde - Babaganoush\n,[None],"Cookbooks, Food & Wine"
581552,22177,11855744,['HI'],"Eggs are a central ingredient in traditional Spanish cuisine, and this omelette recipe combines them with other classic ingredients — potatoes, bell pepper, olive oil, and fresh parsley — for a savory brunch dish.Heat oil and butter in a large, ovenproof nonstick skillet over medium heat. Add potatoes to skillet, spreading them out and flipping them to coat evenly with oil mixture. Cook 6 minutes, turning occasionally, until lightly golden and barely soft. Add onion, bell pepper, garlic, 1/2 teaspoon of the salt, and 1/4 teaspoon of the pepper. Carefully stir into potatoes. Cover skillet; cook 3 minutes, tossing once or twice, until onions are translucent.In a large bowl, whisk eggs with remaining salt and pepper and stir in parsley. Pour over potato mixture. Allow eggs to set in skill...","[HI, re]","Cookbooks, Food & Wine"
623497,64122,13967760,['HI'],"Coleslaw bean sprouts\t1.5\t1 cup Even if you aren’t vegetarian, but are just looking to implement a keto Meatless Monday, or are looking for more keto vegetable side dishes – you’ll find so many delicious and healthy choices in this collection!Lunch: Cajun Tempeh Chicken Salad. Serve with Spicy Creole Gumbo with Collard Greens. Vegans, and anyone who cares about the animals, people, and the environment, do not use palm oil.Spend lots of time researching online. But not seen anything so “to the point”. 2.0 out of 5 starsDisappointingDownload Bonus content Instead of meat, cheese, and dairy, you can have foods like: Condiments and Other Spices So, it’d look a little something like this:5 Reasons to Use MCT Oil for Ketosis 84,147 views Avocados are considered a superfood, because of the ...",[HI],"Cookbooks, Food & Wine"
413515,75991,42627718,['HI'],"Today’s newest recipe post is one of The Sous Chef’s favorites: Mast-o-musir, which is a delicious combination of Lebni and shallots. Must-o-musir literally means yogurt and shallots in Farsi.As you may have noticed there is a lot of memory associated with a lot of food that I love eat and cook. I love sharing, so here is another one for you all!Every time I make or eat this dip it takes me back to the first time I was introduced to it as well as the first time The Sous Chef was introduced to it.In the early eighties I was in elementary school and a kid who was used to the free life in Italy but lived where whatever happened at home remained at home. So if there were parties, gatherings, and a

## Medicine & Health Sciences

In [14]:
label = 'Medicine & Health Sciences'
df = df_all[df_all['genre_prediction'].apply(lambda x: label in x)]
df = df.sample(frac = 1)  # shuffle
df.head(30)

,Unnamed: 0,id,labels,text,register_prediction,genre_prediction
717519,46229,29868749,['IN'],"Relaxation of muscles causing the walls of the upper airway to fall together, causing them to vibrate. Mouthwashes 83+ sold Erectile dysfunction Sleep in women Drug Basics & Safety Register for SDBCC Blood Cancer Symposium & GP talk Ability to Breathe through Mouth? : Yes SSL Certificate Wipes & Hand Sanitizers They deliver pressurized air, which is kept at a constant pressure, to your throat which prevents it from collapsing. As a result, you are able to breathe normally and snore-free through the night. Water Bottles Our triallists said they took a few nights to get used to, though one found it intolerable and dropped out after the first night. The main issues were difficulty breathing though the nose, and a dry and/or sore mouth or throat from mouth-breathing. Two reported a whistli...","[IN, dtp]",Medicine & Health Sciences
171304,57514,47356561,['IN'],"First, the case must involve a “substantial federal interest.” Second, the “prior prosecution must have left that interest demonstrably unvindicated.” Under the policy, this requirement may be met when the defendant was not convicted in the prior proceeding because of incompetence, corruption, intimidation, or undue influence; court or jury nullification in clear disregard of the law; the unavailability of significant evidence; or when the sentence imposed in the prior proceeding was “manifestly inadequate in light of the federal interest involved.” Prosecutions that fall within the policy must be approved in advance by an Assistant Attorney General. He's been on Fludrocortisone (purchased from compounding pharmacy on-line) for 3 yrs now. Specifically, endep 5mg used serotonin modulato...",[IN],Medicine & Health Sciences
158298,44508,46711601,['OP'],"My health stems not only from bad eating, but a difficult past as well. I was involved in three major car accidents, extended hospital stays and surgeries. I lost my sister to cancer and my best friend in one of my car accidents. I had fertility problems, an ectopic pregnancy and lost a child when he was 2 months old due to a heart defect. These resulted in a lot of stress on my system, which created the perfect health problems including headaches, inflammation, weight gain, anxiety and fatigue. We all have a story, and you probably don't realize how much your history can be a direct factor in your health.Why do I share all of this? Let's face it, life can be stressful... kids, spouses, work, finances, sickness, etc. I realized I had to find out what would work best for my system. I al...",[OP],Medicine & Health Sciences
813961,30090,1518576,['IN'],103 epreuve orale dadmission concours as ap no short description epreuve orale dadmission concours as ap because this is pdf file * PDF *\n,[IN],Medicine & Health Sciences
444456,106932,44162340,['IN'],"The typical renal artery is a lateral branch of the aorta, arising at the level of the second lumbar vertebrae, just inferior to the origin of the superior mesenteric artery. In most individuals, each kidney is supplied by one artery. The longer right renal artery (RRA) travels inferolaterally and posterior to the inferior vena cava (IVC) to reach the right renal hilum, while the left renal artery travels horizontally and laterally to reach the left renal hilum. The renal veins are usually anterior to the renal arteries.[1]However, renal vessels are known for presenting a wide range of variations [1] as illustrated by previous cadaveric and imaging studies.[2],[3] These include accessory renal arteries, prehilar branching, and unusual origin of the renal artery which may be proximal to...","[IN, ra]",Medicine & Health Sciences
66506,66506,36496066,['NA'],"""As researchers from both sides of the debate over two controversial H5N1 studies weighed in [Tuesday] on full publication versus a more cautionary approach, two U.S. journals"" -- the Journal of Infectious Diseases (JID) and its sister pub

## Science & Math

In [15]:
label = 'Science & Math'
df = df_all[df_all['genre_prediction'].apply(lambda x: label in x)]
df = df.sample(frac = 1)  # shuffle
df.head(30)

,Unnamed: 0,id,labels,text,register_prediction,genre_prediction
394381,56857,41675273,['NA'],"Farmers in northern Oregon were looking forward to harvest this year. After a dry spring and fading hopes of an above-average crop, their soft, white wheat fields started to show promise this summer. As harvest kicked off, and yields began to roll in, and it was a bountiful wheat crop that came as a surprise to farmers in Wasco County, Ore.The Georgia House of Representatives, led by Rep. Lynn Smith of Newman, won the first annual Legislative Livestock Showdown at the Georgia National Fair in Perry, Oct. 8. Co-sponsored by the Georgia Department of Agriculture and the Georgia National Fair, this event brought together members of the Georgia General Assembly with members of Georgia's FFA and 4-H organizations.""This is one element of the Department of Agriculture's outreach with our stat...","[NA, ne]",Science & Math
728179,56889,30403442,['IN'],"Imprint | Kleffmann Group is the leading provider of agricultural market research & consulting services, delivering key insights for decision makers worldwide.With 25 years of experience, Kleffmann Group is the leading provider of agricultural market research services. As a full service research agency, we are able to provide our clients with key information on multiple agricultural sectors:From over 200,000 interviews per year with farmers and agricultural experts worldwide, we draw up-to-date market knowledge allowing us to offer fast and flexible services with deep insights for both local and global decision makers.Via our amis® panel surveys, we obtain farmer-based information for all major crops worldwide - covering seed as well as crop protection inputs. amis® is comprised of bot...","[IN, dtp]",Science & Math
649658,90283,15278005,['IN'],"The Think Different has a great reputation for stunning yields and very strong weed. Even first time growers have achieved over 100 g/plant and experienced growers can achieve 200g - 300g. It will grow over 1m tall in experienced hands and is a good alternative to traditional varieties, with flowering in just 10 - 11 weeks. The indoor yield can reach a massive 500 g/m2.Dutch Passion Seeds Think Different Auto feminized cannabis seeds are the real surprise package in the Dutch Passion AutoFem collection. The plant has quickly established a great reputation for stunning yields and very strong weed. Even first time growers have achieved over 100 g/plant and experienced growers can achieve closer to 200g - 300g. She can grow over 1m tall in experienced hands and is a good alternative to tr...","[IN, IP, ds]",Science & Math
393623,56099,41638103,['IN'],"Fishing Denise and Rick Fox out from Wisconsin Hi there this is Captain Brett Isackson and this is your daily Peacock bass fishing report for bassCincinnati Reds Baseball Fan Hey, good afternoon everybody this is Captain Roger calling in for another fishing report today. Today on Columbus dayPeacock Bass on a Fly Rod Hi there, this is Captain Brett Isackson this is your daily fishing report.Today I was out what Mike Whalen and his sonThe Emphasis Peacock Bass in Florida Hey everybody this Captain Dave Lauer calling in, today is Friday September 26. I think, I don't even know whatTony Masiello South Florida Peacock Bass Report Hey everybody, this is Tony Masiello in South Florida with today South Florida Peacock bass fishingAlex Adams from Georgia retired Marine Corps Hey, good afternoo...",[NA],Science & Math
770130,98840,32476193,['IN'],If the specified instruction passes execution to the next instruction then the next instruction is returned. Otherwise the lowest referenced address is returned (remember that xrefs are kept sorted!)This is a short fast ride down some back roads. I call it the runway ride because it runs parallel to the runway which i fly planes from. a couple of guys from around here showed me the route and I go down this route when ever I wanna go out for a fast ride.This is a motorcycle r

## No genre prediction

In [17]:
label = 'None'
df = df_all[df_all['genre_prediction'].apply(lambda x: label in x)]
df = df[df['register_prediction'].apply(lambda x: "ID" in x)]
df = df.sample(frac = 1)  # shuffle
df["text"].head(30)

170607    The funniest point I saw her do was to charge a neighbor’s Doberman sending him dwelling worried to Dying. She was my adore for 17 many years. I’ve acquired two Siamese kittens and I hope These are like her.I concur DeannaD. I Enjoy it Risk-free and hold my vital oils (all of that happen to be top rated quality) set away Except I am leaving the world in which my cat hangs out and I realize he won’t be in contact with my pores and skin right until I clean up it off. I remotely use peppermint essential oil really diluted with drinking water in the spray bottle to prevent insects in spots where by the cat doesn’t Regular until finally it dries.Among the list of wild tribes from the Malay Peninsula has domesticated cats, and I've found the woman suckling bengalensis kittens, but I have no ...
753840                                                                                                                                                                                        

## Other way?

In [ ]:
label = 'LY'
df = df_all[df_all['register_prediction'].apply(lambda x: label in x)]
df = df.sample(frac = 1)  # shuffle
df.head(20)

In [ ]:
label = 'ne'
df = df_all[df_all['register_prediction'].apply(lambda x: label in x)]
df = df.sample(frac = 1)  # shuffle
df.head(20)